# An introduction to Optuna
This tutorial guides you through the basic use of Optuna, a Python library for automatic hyperparameter optimization in machine learning. The documentation of Optuna can be found [here](https://optuna.readthedocs.io/en/stable/index.html). 

## Optuna concepts
Three fundamental concepts of Optuna are introduced to help you understand how Optuna works.

**1. Objective function**: An objective function is a user-defined function that takes an Optuna `trial` as its argument. For now, it is enough to understand that a trial is a single round of a hyperparameter optimization process. More detailed explanation will be provided later. An objective function specifies the configuration of hyperparameters to be tuned and returns an evaluation score. An example of an objective function is shown below:
```python
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "criterion": trial.suggest_categorical("criterion", ["gini", "entropy"]),
        "max_depth": trial.suggest_int("max_depth", 1, 10),
        "min_samples_split": trial.suggest_float("min_samples_split", 0.1, 1)
    }
    
    model = RandomForestClassifier(**params)
    model.fit(train_x, train_y)

    predictions = model.predict(test_x)
    accuracy = accuracy_score(y_true=test_y, y_pred=predictions)
    return accuracy
``` 
As you can see from the example, an objective function typically consists of three parts:
1. The search space of each hyperparameter to be tuned. For example, for the `n_estimators` hyperparameter, it is an integer parameter and its value is between 100 and 1000. Similarly, for the `criterion` hyperparameter, it is a categorical parameter and its value is chosen from a fixed list. 
2. Model training. In the example, an Sklearn RandomForest model is trained.
3. Model evaluation (or hyperparameter evaluation). In the example, the model is evaluated using the accuracy score (i.e., the fraction of correct predictions) and the score is finally returned. 

**2. Trial**: Previously, a trial has been simply mentioned as a single round of a hyperparameter optimization process. More specifically, it is a single execution of an objective function. In each trial, for each hyperparameter defined in the objective function, Optuna selects a value from the search range, evaluates the selected hyperparameter combination and return the evaluation metric.

**3. Study**: A study, consisting of multiple trials, refers to an entire hyperparameter optimization process. A study determines the next combination of hyperparameters to be evaluated and keeps track of the trials' results (i.e., the selected hyperparameter combinations and the resulted evaluation metrics). 
```python
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.TPESampler(seed=42),
    storage="sqlite:///optuna_tutorial.sqlite3"
)
study.optimize(objective, n_trials=10)
```
In the example, a study is created. As the `direction` parameter indicates, the target of the study is to maximize the objective value (i.e., the accuracy) returned by the objective function. The `sampler` parameter receives an Optuna `Sampler` object. A `Sampler` decides how to explore and exploit the search space for each hyperparameter based on the results of previous trials to efficiently find better hyperparameter combinations. In the example, the study uses the `TPESampler` that uses TPE (Tree-structured Parzen Estimator) algorithm to determine how to choose hyperparameter values for a new trial. 

When the `study.optimize()` is called, an optimization process is started. The objective function and the number of trials are configured inside the `study.optimize()` function. Optionally, the storage, where the study results are persisted for later analysis, is also specified inside the function. Optuna uses a relational database to store study results. In the example, the database is a simple SQLite file.

<details>
    <summary>What is SQLite? </summary>
    [SQLite](https://www.sqlite.org/index.html) is a lightweight relational database management system. It allows you to create a database as a file, so you can manage and interact with a database without the need for a separate database server. 
</details>

*More reading material: We are not going to the details of the sampling algorithms used by Optuna. For your interest, more details of the sampling algorithms can be found [here](https://optuna.readthedocs.io/en/stable/reference/samplers/index.html).*


## Optuna Examples
You will see three examples that demonstrate the use of Optuna. The [breast cancer dataset](https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic) is used in all of the three examples. 

In these examples, you will train ML models to classify whether a breast tumour is malignant or benign based on certain features of the tumour. Then you will use Optuna to find the optimal hyperparameter combination for the models. 
 

In [ ]:
import optuna
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC 
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import numpy as np
from IPython.display import display

In [ ]:
# Data preparation
X, y = load_breast_cancer(return_X_y=True, as_frame=True)
print(f"Feature shape: {X.shape}")
display(X.head())
print(f"Target shape: {y.shape}")
print(y[:5])

In [ ]:
# Percentage of malignant (0) benign (1) tumours
y.value_counts() / len(y)

In [ ]:
# Use a fixed random seed for reproducibility
RANDOM_SEED = 42

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.25, random_state=RANDOM_SEED)

### Example 1: The basic use of Optuna
In this example, you will train an Sklearn's RandomForest model for the breast tumour classification use case. [Area Under the Receiver Operating Characteristic Curve (ROC AUC)](https://scikit-learn.org/stable/modules/model_evaluation.html#receiver-operating-characteristic-roc) is used for model evaluation. 

The hyperparameters to be configured are listed below:
|Hyperparameter| Explanation|type|
|--------------| -----------|----|
|n_estimators  | The number of trees. | Integer|
|criterion     | The function to measure the quality of a split.|String|
|max_depth     | The maximum depth of the tree.|Integer|
|min_samples_split|The minimum fraction of samples required to further split an internal node|Float|
|random_state|Control the randomness in model training. It's fixed in the example for reproducibility|Integer|


*If you are interested, more explanation of the hyperparameters of the sklearn random forest classification model can be found [here](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html).*

In [ ]:
# Define the objective function
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "criterion": trial.suggest_categorical("criterion", ["gini", "entropy"]),
        "max_depth": trial.suggest_int("max_depth", 1, 10),
        "min_samples_split": trial.suggest_float("min_samples_split", 0.1, 1),
        "random_state": RANDOM_SEED
    }
    
    model = RandomForestClassifier(**params)
    model.fit(train_x, train_y)

    # Predict class probabilities
    predictions = model.predict_proba(test_x)

    # In binary classification, the probability of the class with the “greater label” should be provided. 
    # The “greater label” corresponds to classifier.classes_[1] and thus classifier.predict_proba(X)[:, 1].
    # See https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case
    score = roc_auc_score(test_y, predictions[:, 1])
    return score

In [ ]:
# Create and run a study
study_name = "rf-breast-tumour"
storage = "sqlite:///optuna_tutorial.sqlite3"

study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.TPESampler(seed=RANDOM_SEED),
    study_name=study_name,
    storage=storage,
    load_if_exists=True
)

study.optimize(objective, n_trials=10)

You should see a SQLite database file "optuna_tutorial.sqlite3" appear under the same directory with this notebook. The results of study "rf-breast-tumour" is saved in the "optuna_tutorial.sqlite3" file. 

After the study is completed, you can load the study from the database.

In [ ]:
loaded_study = optuna.load_study(study_name=study_name, storage=storage)

# Check the best trial
print(f"The best roc_auc_score: {loaded_study.best_value}")
print(f"The best hyperparameter combination: {loaded_study.best_params}")

Optuna also provides functionalities for visualizing a study. For example, you can plot a figure to see how much a hyperparameter matters to the objective value (i.e., the roc_auc_score in this case).

*If you are interested, more documentation of the Optuna's visualization functionalities can be found [here](https://optuna.readthedocs.io/en/stable/reference/visualization/index.html).*

In [ ]:
import plotly.io as pio

# Configure Jupyter Notebook to render plotly figures drawn by Optuna
pio.renderers.default = "notebook"

Different algorithms can be used to evaluate hyperparameter importance. The default one used by Optuna
is the [*fANOVA importance evaluator*](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.importance.FanovaImportanceEvaluator.html#optuna.importance.FanovaImportanceEvaluator). The fixed `RANDOM_SEED` is assigned to the evaluator's seed for reproducibility. 

In [ ]:
fig = optuna.visualization.plot_param_importances(loaded_study, evaluator=optuna.importance.FanovaImportanceEvaluator(seed=RANDOM_SEED))
fig.show()

Please note that once a study is already stored in a database, an error will be raised if you try to record another study with the same name. There are two workarounds for this issue:
1. Explicitly setting the argument `load_if_exit=True` when calling `optuna.create_study()`. This will cause Optuna to append the new results to the existing ones for that study.
2. Delete the existing study using the code provided in the next code cell.

In this week's tutorial and assignments, we choose the second approach to ensure reproducibility. 

In [ ]:
# Uncomment the code below if you want to delete the study in the database

# study_name = "rf-breast-tumour"
# storage = "sqlite:///optuna_tutorial.sqlite3"
# optuna.delete_study(study_name=study_name, storage=storage)


### Example 2: Tuning hyperparameters for multiple models -- Conditional search spaces with Optuna
In some use cases, you may want to compare the performance of different model libraries or model structures. In these cases, Optuna can be used to optimize the hyperparameters for multiple models in a single study.

In this example, you will see how to tune hyperparameters for multiple models using Optuna, where the hyperparameters to be tuned, their search spaces and the model fitting are customized based on different model types. Specifically, we'll tune a logistic regression model and a random forest model and see which one performs better in the breast cancer use case. The hyperparameters to be configured for the random forest model are the same as Example 1. The hyperparameters of the logistic regression model are listed below.
|Hyperparameter| Explanation|type|
|--------------| -----------|----|
|penalty  | Type of regularization to apply to the model. | String|
|C     | Control the strength of the regularization. Smaller value leads to stronger regularization|Float|
|solver     | The algorithm used to minimize the cost function. It's fixed in the example|String|
|max_iter|Maximum number of iterations taken for the solver to converge.|Integer|
|random_state|Control the randomness in model training. It's fixed in the example for reproducibility|Integer|

*If you're interested, more details of the hyperparameters for an sklearn logistic regression model can be found [here](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html).*


In [ ]:
def objective(trial):
    
    # We treat the model type as a hyperparameter. In this way, Optuna will perform more trials experimenting with the more promising model type.
    classifier_name = trial.suggest_categorical("classifier", ["logit", "rf"])
    
    # When the model is a logistic regression model
    if classifier_name == "logit":
        params = {
            "penalty": trial.suggest_categorical("penalty", ["l1","l2"]),
            "C": trial.suggest_float("C", 0.001, 10),
            "solver": "saga",
            "max_iter": 1000,
            "random_state": RANDOM_SEED
        }
        
        # Data standardization may improve accuracy of a logistic regression model
        model = make_pipeline(StandardScaler(), LogisticRegression(**params))
        
    # When the model is a random forest model
    elif classifier_name == "rf":
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
            "criterion": trial.suggest_categorical("criterion", ["gini", "entropy"]),
            "max_depth": trial.suggest_int("max_depth", 1, 10),
            "min_samples_split": trial.suggest_float("min_samples_split", 0.1, 1),
            "random_state": RANDOM_SEED
        }
    
        model = RandomForestClassifier(**params)

    model.fit(train_x, train_y)
    predictions = model.predict_proba(test_x)[:, 1]
    score = roc_auc_score(test_y, predictions)
    return score

In [ ]:
study_name = "multimodel-breast-tumour"

study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.TPESampler(seed=RANDOM_SEED),
    study_name=study_name
)
study.optimize(objective, n_trials=10)
print(f"The best roc_auc_score: {study.best_value}")
print(f"The best hyperparameter combination: {study.best_params}")


### Example 3: Ensemble averaging 
In some use cases, it may be useful to combine the outputs of multiple models to make the final prediction (ensemble models). One approach to make the final decision is to assign a weight to each model based on its performance, and these weights are used to determine the contribution of each model in the final prediction. For example, suppose there are $n$ models, whose predictions are denoted by $P_1$, $P_2$, ..., $P_n$, the models are assigned a weight of $w_1$, $w_2$, ..., $w_n$, respectively, the final prediction can then be calculated as:
$$ final\_prediction = {\sum_{i=1}^{n}w_iP_i \over \sum_{i=1}^{n}w_i} $$

In regression use cases, the *final_prediction* is simply the final result. In classification, the *final_prediction* can be the probability estimates, i.e., the likelihood of the input belonging to a class. 

Optuna can be used to find a good weight combination for combining the models. 

First, train three separate models and evaluate them. For simplicity, we just use default configurations for all hyperparameters (except for `random_state` which is set as a fixed value for reproducibility). 

In [ ]:
# random forest
rf_model = RandomForestClassifier(random_state=RANDOM_SEED)

# logistic regression
logit_model = make_pipeline(StandardScaler(), LogisticRegression(random_state=RANDOM_SEED)) 

# support vector classification
svc_model = make_pipeline(StandardScaler(), SVC(random_state=RANDOM_SEED, probability=True))

model_names = ["rf_model", "logit_model", "svc_model"]

for name in model_names:
    model = eval(name) # The eval() function evaluates a specified expression, if the expression is a legal Python statement, it will be executed.
    model.fit(train_x, train_y)
    predictions = model.predict_proba(test_x)[:, 1]
    score = roc_auc_score(test_y, predictions)
    print(f"{name}: {score}")


The best roc_auc_score is 0.9977 from the logistic regression model.

Next, use Optuna to find a good weight combination to combine the outputs of these three models and see if this can improve the final roc_auc_score. 

In [ ]:
# This is a dictionary where each key is a model name, 
# and the corresponding value is an array of the predicted probabilities (for the class with the greater label) made by the model.
all_predictions = {name: (eval(name)).predict_proba(test_x)[:, 1] for name in model_names}

def objective(trial):
    # This is a dictionary where each key is a model name, 
    # and the corresponding value is the weight assigned to the model.
    weights = {name: trial.suggest_int(name, 1, 100) for name in model_names}

    # [weights[name] * all_predictions[name] for name in model_names] forms a list of arrays, 
    # where each array is the weighted predictions of a model. This is done for each model specified in model_names.
    # Then we use np.sum(..., axis=0) to sum up weighted predictions across all models for each data point. This produces an array.
    # Finally, we divide the array by the sum of weights.
    probs = np.sum([weights[name] * all_predictions[name] for name in model_names], axis=0)/sum(weights.values())
   
    score = roc_auc_score(test_y, probs)
    return score

In [ ]:
study_name = "ensemble-breast-tumour"

study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.TPESampler(seed=RANDOM_SEED),
    study_name=study_name
)
study.optimize(objective, n_trials=20)

print(f"The best roc_auc_score: {study.best_value}")
print(f"The best weight combination: {study.best_params}")

The best roc_auc_score after combining the predictions of the three models is 0.9979, which is slightly better than the roc_auc_score of the best singe model (0.9977).

## Parallelization in Python
One way to speed up hyperparameter optimization is multi-processing: utilizing multiple processes to evaluate different sets of hyperparameters simultaneously. One way to implement multi-processing in Python is to use the joblib package, a Python library used for efficient parallel computing (and data serialization.) 

Below, you will see an example of using the joblib package to implement multi-processing. The example shows the basic use of the joblib's parallel computing functionalities but doesn't touch Optuna. You will use the joblib package to parallelize an Optuna study in one of this week's assignments. 


Before going to the use of joblib, let's define a simple function that calculates the square root of a given number.

In [ ]:
import time, math
def my_square_root(number):
    # Wait for 1 second before starting the calculation
    time.sleep(1)
    return math.sqrt(number)

First, we run the function for 10 times sequentially, without parallelization.

In [ ]:
iteration_count = 10
start = time.time()
for i in range(iteration_count):
    my_square_root(i)
end = time.time()
print(f"Consumed time: {end - start}s.")

Since the `my_square_root` function needs to wait for 1s before starting the calculation, it takes around 10s to run this function 10 times. 

Next, let's run the function 10 times again, but we parallelize these 10 runs using 2 processes at this time. The [joblib.Parallel](https://joblib.readthedocs.io/en/latest/generated/joblib.Parallel.html) class is needed to implement multi-processing. 

In [ ]:
from joblib import Parallel, delayed

start = time.time()
Parallel(n_jobs=2)(delayed(my_square_root)(i) for i in range(iteration_count))
end = time.time()
print(f"Consumed time: {end - start}s.")

Let's break down the code `Parallel(n_jobs=2)(delayed(my_square_root)(i) for i in range(iteration_count))` into smaller parts.

- `Parallel(n_jobs=2)`: The `Parallel` class is used to set up a parallel processing context with two processes (n_jobs=2).

- `delayed(...)`: The `delayed` function takes a function (`my_square_root` in the example) and generates a "delayed" task of it. In other words, this "delayed" task of the function will be executed later in parallel instead of being executed immediately. 

- `delayed(my_square_root)(i) for i in range(iteration_count)`: The `delayed` function generates `iteration_count` (which is 10 in the example) tasks to be executed in parallel, each calling the `my_square_root` function with arguments from 0 to `iteration_count-1`. 

- `Parallel(...)(...)`: The Parallel object returned from `Parallel(n_jobs=2)` is invoked to perform the delayed tasks generated by the `delayed` function. The multi-processing computation is started.

Since the 10 runs of the `my_square_root` function are parallelized in two processes, the consumed time is around halved compared to the non-parallelized version. (The time is always a bit more than exactly half of the initial time because it takes time for the processes to communicate and coordinate with each other.)